# IBM Professional Data Science Certificate - Capstone Project

In [1]:
import pandas as pd
import numpy as np 

print("Hello Capstone Project Course!")

Hello Capstone Project Course!


## Introduction: Business Problem
**Fitness** is a major global industry. It's estimated to generate at least $100 billion world wide. And, in the US over the years 2013-2017 there was a drastic in boutique gym studios, which grew memberships by 121%. Interestingly, people who exercise also generate are often higher in demand as workers and generate higher salaries. As such, they're part of a high-demand, and relatively mobile workforce. This target demographic of people will also make living and working arrangements to be close to fitness studios. 

We also live in a dystopian parallel universe where Yelp and similar services don't exist. We need a way to find out which neighborhoods have the highest prevalence of fitness studios. So, we will use our fancy data science skills to solve this problem. We will help fitness-conscious people in the workforce find neighborhoods to live in Toronto that have a higher prevalence of fitness studios that they have access to to support their lifestyle.

## Data Description
The data required to solve this business problem at a high-level can be leveraged from the datasets used previously in this capstone course. ** Wikipedia **, the **Geospatial Coordiates** `CSV` file, and the **Four Square API** will be leveraged. Popular neighborhoods will be identified geospatially through a combination of the Wikipedia tables and the `CSV` file. Venues of type `fitness studio` will be pulled for those neighborhoods. And, we will determine which neighborhoods have the highest frequency of fitness studios.

